In [4]:
import requests as r
from bs4 import BeautifulSoup as BS

In [5]:
directory_page_link: str = 'https://www.bu.edu/academics/degree-programs/'

base_url: str = 'https://www.bu.edu'

In [6]:
page = BS(r.get(directory_page_link).text)

In [7]:
degrees_information = page.find(id="post-30760", class_="content-panel").find_all('li')

In [8]:
from IPython.display import clear_output

degrees = {}

nd = degrees_information.__len__()

for i, di in enumerate(degrees_information):

    clear_output()

    print(f'{i+1}/{nd}')

    allow_degrees = ['BS', 'BA', 'minor'] # , 'BS/MS', 'BA/MA', 'BS/MA', 'BA/MS', ]

    name = di.text.split(' (')[0]

    offers = di.find_all('a')

    links = [o['href'] for o in offers]

    degrs = [o.text for o in offers]

    pages = []

    for l in links:

        try:
            pages.append(r.get(base_url + l).text)

        except:

            pages.append(None)

    format_offers = [{"Degree": deg, 'Link': link, 'Page': page} for deg, link, page in zip(degrs, links, pages) if deg in allow_degrees and page is not None]

    if format_offers.__len__() != 0:

        degrees[name] = {
            'name': name,
            'offers': format_offers
        }

506/506


In [9]:
degrees.__len__()

166

In [10]:
degrees['Advertising COM']

{'name': 'Advertising COM',
 'offers': [{'Degree': 'BS',
   'Link': '/academics/com/programs/advertising/advertising-bs/',
   'Page': '<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html class="ie6 " lang="en"> <![endif]-->  \n<!--[if IE 7 ]> <html class="ie7 " lang="en"> <![endif]-->  \n<!--[if IE 8 ]> <html class="ie8 " lang="en"> <![endif]-->  \n<!--[if IE 9 ]> <html class="ie9 " lang="en"> <![endif]-->  \n<!--[if IE 10 ]> <html class="ie10 " lang="en"> <![endif]--> \n<!--[if !IE]><!--> <html class="notIE" lang="en"> <!--<![endif]-->  \n<head>\n    \n    <title>BS in Advertising &raquo;  Academics  | Boston University</title>\n    \n    <!--meta cluster-->\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:[]}};(window.NREUM||(NREUM={})).loader_config={xpid:"UgUAV1RQGwQCVVNXAwY=",licenseKey:"b19c58809e",applicationID:"7212547"};;/*! For l

In [11]:
[degree for degree in degrees if degrees[degree]['offers'].__len__() > 1]

['Advertising COM',
 'African American & Black Diaspora Studies',
 'American Studies ',
 'Ancient Greek',
 'Archaeological & Environmental Sciences',
 'Archaeology',
 'Art & Architecture, History of',
 'Astronomy',
 'Biology CAS/GRS',
 'Biomedical Engineering',
 'Cinema & Media Studies',
 'Classical Civilization',
 'Comparative Literature',
 'Computer Engineering',
 'Computer Science CAS/GRS',
 'Data Science',
 'Deaf Studies',
 'Earth & Environmental Sciences',
 'Economics CAS/GRS',
 'Electrical Engineering',
 'English',
 'English Education',
 'Environmental Analysis & Policy',
 'European Studies',
 'Film & Television',
 'French Studies',
 'Greek—Ancient',
 'History CAS/GRS',
 'History of Art & Architecture',
 'Holocaust, Genocide & Human Rights Studies',
 'Hospitality Administration',
 'Human Physiology',
 'Interdisciplinary Studies',
 'International Relations',
 'Japanese Language & Literature',
 'Journalism',
 'Latin',
 'Latin American Studies',
 'Linguistics CAS/GRS',
 'Marine Scie

In [12]:
# All pages have h2 with required or requirement in it

sum([[degree + ' ' + degrees[degree]['offers'][i]['Degree'] for i in range(len(degrees[degree]['offers'])) if not BS(degrees[degree]['offers'][i]['Page']).find(lambda tag: tag is not None and tag.name == 'h2' and 'requirement' in tag.text.lower() or 'required' in tag.text.lower())] for degree in degrees], [])

[]

In [ ]:
# Using selenium 
# 1. sign in 
# 2. go to the what if page 
# 3. generate reports for a degree then scrape req categories 
# 4. if sub categories iterate to those
# 5. Iteravley click view courses scrape then exit

In [ ]:
%pip install selenium webdriver-manager


In [50]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import requests
from dotenv import load_dotenv
import os, time
from time import sleep

from bs4 import BeautifulSoup as BS

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

create_report_button = "//a[./span[text()='Create Report']]"
add_scenario_button = "//button[./span[text()='Add Scenario']]"
second_dropdowns = "//div[./label[text()='{}']]"
drop_down_options = "//div[@role='presentation']"

driver = None

def scraper():

    global driver
    
    # Step 1: Set up Selenium with headless mode
    options = Options()

    service = Service(ChromeDriverManager().install())

    driver = webdriver.Chrome(service=service, options=options)

    def login():
        load_dotenv()
        username_str = os.getenv('BU_USERNAME')
        password_str = os.getenv('BU_PASSWORD')

        #enter username
        username = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "j_username"))
    )
        username.send_keys(username_str)

        #enter password
        password = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "j_password"))
        )
        password.send_keys(password_str)

        #click login button
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "input-submit"))
        )
        button.click()
        return current_url
    
    def duo_mobile():
        #click login button
        print('******DUO MOBILE BLOCKED. Please open your app and approve.******')
        button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.ID, "trust-browser-button"))
        )
        button.click()
        print('Duo passed. Collecting data...')
         # Wait for the URL to change
        try:
            WebDriverWait(driver, 10).until(EC.url_changes(driver.current_url))
        except:
            print("URL did not change within the expected time.")
        
    # Step 2: Automate login
    login_url = "https://shib.bu.edu/idp/"
    class_search_url="https://mybustudent.bu.edu/psp/BUPRD/EMPLOYEE/SA/s/WEBLIB_HCX_RE.H_WHAT_IF.FieldFormula.IScript_Main?"
    driver.get(class_search_url)
    current_url = driver.current_url
    if current_url.__contains__(login_url):
        current_url = login()

    # Wait for the URL to change
    try:
        WebDriverWait(driver, 10).until(EC.url_changes(current_url))
    except:
        print("URL did not change within the expected time.")

    # Check if you need Duo Mobile bot
    new_url = driver.current_url
    if new_url.__contains__('duosecurity'):
        duo_mobile()
    
    # Step 3: Extract cookies from the Selenium session
    cookies = driver.get_cookies()
    session_cookies = {cookie['name']: cookie['value'] for cookie in cookies}   

    iframe = driver.find_elements(By.TAG_NAME, "iframe")[0]


    driver.switch_to.frame(iframe)

    get_degree_req(None, None, driver)

    sleep(10)

    # Clean up
    driver.quit()

def get_degree_req(name, degree, driver):

    # Start at what if page

    # 1. Click on create report

    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, create_report_button))
    )
    button.click()

    # 2. Click on Add Senario

    # 2a. Scroll to make Add Scenario Visible
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, add_scenario_button))
    )

    driver.execute_script("arguments[0].scrollIntoView();", button)

    button.click()

    # Click on Dropdowns from Program -> Plan Type -> Plan

    # Functions

    # Find dropdown to click on
    get_dropdown_click = lambda name: driver.find_elements(By.XPATH, second_dropdowns.format(name))[1].find_elements(By.TAG_NAME, 'svg')[0]

    # Find dropdown option by text
    def get_option(text: str):

        wait = WebDriverWait(driver, timeout=2)

        dropdown = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='presentation']")))

        option_to_select = dropdown.find_element(By.XPATH, f"//li[text()='{text}']")
        option_to_select.click()


    # Select the program

    get_dropdown_click('Program').click()

    get_option('Arts & Sciences UGRD')

    get_dropdown_click('Plan Type').click()

    get_option('Minor')

    get_dropdown_click('Plan').click()

    get_option('Biology MIN')

    # 3. 

try:
    scraper()
except Exception as e:

    print(e)

    driver.quit()


******DUO MOBILE BLOCKED. Please open your app and approve.******
Duo passed. Collecting data...


In [17]:
degrees['Advertising COM']

{'name': 'Advertising COM',
 'offers': [{'Degree': 'BS',
   'Link': '/academics/com/programs/advertising/advertising-bs/',
   'Page': '<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html class="ie6 " lang="en"> <![endif]-->  \n<!--[if IE 7 ]> <html class="ie7 " lang="en"> <![endif]-->  \n<!--[if IE 8 ]> <html class="ie8 " lang="en"> <![endif]-->  \n<!--[if IE 9 ]> <html class="ie9 " lang="en"> <![endif]-->  \n<!--[if IE 10 ]> <html class="ie10 " lang="en"> <![endif]--> \n<!--[if !IE]><!--> <html class="notIE" lang="en"> <!--<![endif]-->  \n<head>\n    \n    <title>BS in Advertising &raquo;  Academics  | Boston University</title>\n    \n    <!--meta cluster-->\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:[]}};(window.NREUM||(NREUM={})).loader_config={xpid:"UgUAV1RQGwQCVVNXAwY=",licenseKey:"b19c58809e",applicationID:"7212547"};;/*! For l